# Synthetic Biomedical Data – Lesson 3b: Random Effects (Batch / Subject Effects)

Part of the *Microcredit Biomedical Data Generator* learning module.

➡️ [Back to Lesson 3a: Irrelevant Features](03a_irrelevant_features.ipynb)
➡️ [Module Overview (README)](../README.md)

---

### Recap from Lesson 03a Irrelevant Features
You:
- Added **noise features** to a clean baseline.
- Observed chance-driven **false discoveries** in top-k rankings.
- Used cross-validation to quantify the stability of apparent signals.

### Why this lesson: Random effects?

In real biomedical studies, measurements are often collected in **batches** (different plates, days, machines, centers) or **subjects** (repeated measures). These groups induce **random effects**—systematic offsets (and sometimes slopes) that are *not* the biological signal of interest. If we ignore them, we may:

- overestimate effect sizes,
- leak information across cross-validation folds,
- pick the wrong "top features",
- and deploy models that fail in the wild.

This notebook introduces random effects using controlled synthetic data, showing how they interact with class labels, how they distort metrics, and how to diagnose them early with the right CV design.

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Simulate** batch/subject random effects on top of a clean informative dataset.
2. **Visualize** how random intercepts shift feature distributions across batches.
3. **Quantify** the difference between *naïve* effect sizes and *batch-corrected* effect sizes.
4. **Prepare** the evaluation for group-aware cross-validation (e.g., `GroupKFold`) to avoid leakage in later steps.


## Prerequisites & Design

We build on the previous notebook (03a) that introduced **noise features**.
Here we keep a small informative core and add **batch-level random intercepts**.

**Key toggles in this notebook:**
- `CONFOUNDED`: if `True`, batches correlate with the class (realistic worst case).
- `BATCH_SD`: magnitude of the batch intercept shift.
- `N_BATCHES`: number of batches to simulate.

We'll first compare **global** effect sizes (ignoring batches) vs. **batch-corrected** effect sizes (demeaning per batch). In a later section, we will revisit model evaluation using **grouped CV**.


In [ ]:
# Reproducibility & plotting
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Synthetic data generator
from biomedical_data_generator import DatasetConfig
from biomedical_data_generator.generator import generate_dataset

# Pretty display
pd.set_option("display.max_rows", 20)
pd.set_option("display.width", 120)

# --- Helpers ---
def cohen_d(x1: np.ndarray, x2: np.ndarray) -> float:
    """
    Compute Cohen's d for two independent samples.
    Uses pooled standard deviation with Bessel's correction.
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)
    n1, n2 = len(x1), len(x2)
    if n1 < 2 or n2 < 2:
        return 0.0
    m1, m2 = x1.mean(), x2.mean()
    s1, s2 = x1.std(ddof=1), x2.std(ddof=1)
    # pooled SD
    s_pooled = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
    return (m1 - m2) / s_pooled if s_pooled > 0 else 0.0


def effect_sizes_by_feature(X: pd.DataFrame, y: pd.Series) -> pd.DataFrame:
    """
    Compute |Cohen's d| for all columns in X between classes y in {0,1}.
    Returns a sorted DataFrame.
    """
    out = {}
    x0 = X[y == 0]
    x1 = X[y == 1]
    for col in X.columns:
        out[col] = abs(cohen_d(x0[col].values, x1[col].values))
    df = pd.DataFrame.from_dict(out, orient="index", columns=["|Cohen_d|"]).sort_values("|Cohen_d|", ascending=False)
    df.index.name = "Feature"
    return df


In [ ]:
# --- Core dataset parameters (keep small and interpretable) ---
N_SAMPLES   = 240
N_INFORM    = 8
N_PSEUDO    = 0
N_NOISE     = 6
N_CLASSES   = 2
CLASS_SEP   = 1.2
RANDOM_SEED = 42

# --- Random-effects parameters ---
N_BATCHES   = 4
BATCH_SD    = 1.0     # magnitude of random intercept per batch
CONFOUNDED  = True    # if True, batches correlate with the class label (worst case)
AFFECTED    = "all"   # "all" or "informative" — which features receive batch offsets

rng = np.random.default_rng(RANDOM_SEED)


In [ ]:
cfg = DatasetConfig(
    n_samples=N_SAMPLES,
    n_informative=N_INFORM,
    n_pseudo=N_PSEUDO,
    n_noise=N_NOISE,
    n_classes=N_CLASSES,
    class_sep=CLASS_SEP,
    feature_naming="prefixed",  # i1, i2, ..., n1, ...
    random_state=RANDOM_SEED,
)

# Return DataFrame for convenience in EDA
X, y, meta = generate_dataset(cfg, return_dataframe=True)

print(f"X shape: {X.shape}, y shape: {y.shape}")
display(X.head(3))

# Feature lists
informative_cols = [c for c in X.columns if c.startswith("i")]
noise_cols       = [c for c in X.columns if c.startswith("n")]
feature_cols     = informative_cols + noise_cols  # (no pseudo in this notebook)


In [ ]:
def assign_batches(y: pd.Series, n_batches: int, confounded: bool, p_major: float = 0.7, seed: int = 0) -> np.ndarray:
    """
    Create a batch assignment for samples.
    If confounded=True, class 1 is overrepresented in batches {0,1} and class 0 in {2,3} (for n_batches=4),
    controlled by p_major.
    """
    rng_local = np.random.default_rng(seed)
    y = y.values.astype(int)
    n = len(y)

    if not confounded:
        return rng_local.integers(0, n_batches, size=n)

    # Simple confounding scheme for n_batches >= 2
    major_for_1 = list(range(min(2, n_batches)))               # e.g., batches 0,1
    major_for_0 = list(range(2, min(4, n_batches))) or [0]     # e.g., batches 2,3 (fallback 0)

    batches = np.empty(n, dtype=int)
    for i, yi in enumerate(y):
        if yi == 1:
            if rng_local.random() < p_major:
                batches[i] = rng_local.choice(major_for_1)
            else:
                others = [b for b in range(n_batches) if b not in major_for_1]
                batches[i] = rng_local.choice(others or major_for_1)
        else:
            if rng_local.random() < p_major:
                batches[i] = rng_local.choice(major_for_0)
            else:
                others = [b for b in range(n_batches) if b not in major_for_0]
                batches[i] = rng_local.choice(others or major_for_0)
    return batches

batch_id = assign_batches(y, n_batches=N_BATCHES, confounded=CONFOUNDED, p_major=0.75, seed=RANDOM_SEED)
X["batch_id"] = batch_id

# Quick sanity checks
print("Class counts:", dict(pd.Series(y).value_counts().sort_index()))
print("Batch counts:", dict(pd.Series(batch_id).value_counts().sort_index()))
print("Contingency (batch × class):")
display(pd.crosstab(X["batch_id"], y))


In [ ]:
# Choose which columns receive batch offsets
if AFFECTED == "informative":
    cols_to_shift = informative_cols
else:
    cols_to_shift = feature_cols

# Draw one intercept per batch (same for all affected features)
batch_intercepts = rng.normal(loc=0.0, scale=BATCH_SD, size=N_BATCHES)

X_shifted = X.copy()
for b in range(N_BATCHES):
    mask = X_shifted["batch_id"].values == b
    X_shifted.loc[mask, cols_to_shift] = X_shifted.loc[mask, cols_to_shift] + batch_intercepts[b]

# Keep original for comparison
X_raw = X[feature_cols].copy()
X = X_shifted  # from here on, X contains random-effects shifts


In [ ]:
# Pick a few representative features to visualize
show_cols = (informative_cols[:3] + noise_cols[:3])[:6]

n_rows, n_cols = 2, 3
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 8), constrained_layout=True)

for i, col in enumerate(show_cols):
    ax = axes[i // n_cols, i % n_cols]
    sns.histplot(
        data=pd.DataFrame({col: X[col], "class_": y, "batch_id": X["batch_id"]}),
        x=col, hue="class_", stat="density", element="step", common_norm=False, alpha=0.3, kde=True, ax=ax,
    )
    ax.set_title(f"{col} — by class (batches mixed)")
    ax.set_ylabel("Density")

plt.show()

# Facet by batch for the first feature to see the intercept shifts clearly
col = show_cols[0]
g = sns.FacetGrid(pd.DataFrame({col: X[col], "class_": y, "batch_id": X["batch_id"]}), col="batch_id", col_wrap=4, sharex=True, sharey=True, height=3)
g.map_dataframe(sns.kdeplot, x=col, hue="class_", common_norm=False, fill=False)
g.add_legend()
g.fig.suptitle(f"Batch-wise KDE for {col}", y=1.02)
plt.show()


In [ ]:
# 1) Naïve (global) effect sizes
es_global = effect_sizes_by_feature(X[feature_cols], y)
display(es_global.head(10))

# 2) Batch-corrected effect sizes via per-batch demeaning (random-intercept removal)
X_bc = X[feature_cols].copy()
for col in feature_cols:
    X_bc[col] = X_bc[col] - X.groupby("batch_id")[col].transform("mean")

es_batch_corrected = effect_sizes_by_feature(X_bc, y)
display(es_batch_corrected.head(10))

# Compare both views side-by-side for the top features
cmp = es_global.join(es_batch_corrected, lsuffix="_global", rsuffix="_batch_corrected")
cmp = cmp.sort_values("|Cohen_d|_global", ascending=False)
display(cmp.head(15))


## Interim Takeaways

- **Random intercepts** can make features look strongly discriminative in a *global* view, especially when batches are **confounded** with the class.
- **Per-batch demeaning** (a simple random-intercept removal) often shrinks those inflated effect sizes.
- This is an EDA-stage warning sign: if rankings flip after batch correction, you likely need **group-aware cross-validation** and possibly **mixed-effects models** or dedicated batch-correction strategies.

**Next:** Evaluate models with **`GroupKFold`** (group = `batch_id`) vs. naïve CV to see real-world impact on performance.

### Why compare two CV schemes now?

We just saw how random intercepts inflate global effect sizes. The next step is to check whether model **evaluation** is also biased. We will run the **same pipeline** under two CV schemes:

- **Naïve StratifiedKFold** (leaky wrt batches)
- **GroupKFold** with `group = batch_id` (out-of-batch generalization)

**Expectation:** If batches are confounded and/or batch SD is large, naïve CV will report **optimistic** scores compared to group-aware CV.

In [ ]:
# Models & CV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GroupKFold, cross_validate

# Use only the feature matrix (no batch_id) for modeling
X_mat = X[feature_cols].values
y_arr = y.values
groups = X["batch_id"].values  # used only in GroupKFold

pipe = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(solver="lbfgs", max_iter=1000, random_state=RANDOM_SEED)),
    ]
)

# Two evaluation schemes: naïve (leaky w.r.t. batches) vs. grouped (batch-safe)
cv_naive   = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
cv_grouped = GroupKFold(n_splits=min(N_BATCHES, 5))

scoring = {"bal_acc": "balanced_accuracy", "roc_auc": "roc_auc"}

# Naïve CV (no grouping)
res_naive = cross_validate(
    pipe, X_mat, y_arr,
    cv=cv_naive,
    scoring=scoring,
    n_jobs=-1,
    return_estimator=False,
)

# Group-aware CV (groups = batch_id)
res_group = cross_validate(
    pipe, X_mat, y_arr,
    cv=cv_grouped,
    groups=groups,
    scoring=scoring,
    n_jobs=-1,
    return_estimator=False,
)

def summarize(res, label):
    return pd.DataFrame(
        {
            "scheme": [label, label],
            "metric": ["balanced_accuracy", "roc_auc"],
            "mean":   [res["test_bal_acc"].mean(), res["test_roc_auc"].mean()],
            "std":    [res["test_bal_acc"].std(ddof=1), res["test_roc_auc"].std(ddof=1)],
        }
    )

summary = pd.concat([summarize(res_naive, "naive"), summarize(res_group, "grouped")], ignore_index=True)
summary


### Visualizing the gap

A compact plot helps to internalize the gap between the two evaluation schemes. If the bar for "naïve" sits clearly above "grouped", you have evidence of **batch leakage**.


In [ ]:
import matplotlib.pyplot as plt

# Bar plot for balanced accuracy means with error bars (±1 SD)
plot_df = summary[summary["metric"] == "balanced_accuracy"].copy()

fig, ax = plt.subplots(figsize=(5, 4))
ax.bar(plot_df["scheme"], plot_df["mean"], yerr=plot_df["std"], capsize=5)
ax.set_ylabel("Balanced accuracy (CV)")
ax.set_title("Naïve vs. Group-aware CV")
for i, v in enumerate(plot_df["mean"]):
    ax.text(i, v, f"{v:.3f}", ha="center", va="bottom")
plt.tight_layout()
plt.show()

# (Optional) Also display ROC-AUC table rounded
display(summary.pivot(index="scheme", columns="metric", values="mean").round(3))


### What do misclassifications look like?

Confusion matrices make the leakage tangible. If the naïve CV predicts much better than the grouped CV, your model likely learned batch structure instead of true biology. This validates the need for **group-aware** evaluation (and later, proper batch handling).

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Naïve predictions
y_pred_naive = cross_val_predict(pipe, X_mat, y_arr, cv=cv_naive, method="predict")
# Group-aware predictions
y_pred_group = cross_val_predict(pipe, X_mat, y_arr, cv=cv_grouped, groups=groups, method="predict")

print("Naïve CV — classification report")
print(classification_report(y_arr, y_pred_naive, digits=3))

print("\nGroup-aware CV — classification report")
print(classification_report(y_arr, y_pred_group, digits=3))

fig, axes = plt.subplots(1, 2, figsize=(10, 4), constrained_layout=True)
ConfusionMatrixDisplay(confusion_matrix(y_arr, y_pred_naive)).plot(ax=axes[0], colorbar=False)
axes[0].set_title("Naïve CV")

ConfusionMatrixDisplay(confusion_matrix(y_arr, y_pred_group)).plot(ax=axes[1], colorbar=False)
axes[1].set_title("Group-aware CV")
plt.show()


## Recap & Interpretation

- **Random intercepts** shift feature distributions per batch; when confounded with the class, both **effect sizes** and **naïve CV** are inflated.
- **GroupKFold** approximates out-of-batch generalization and typically reduces overly optimistic metrics.
- **EDA signals to watch:**
  - Feature ranks change after per-batch demeaning.
  - Naïve > grouped CV, often by a sizeable margin.
- **Actionable next steps** (not all implemented here):
  - Always use **group-aware CV** when batches/subjects exist.
  - Consider **batch-correction** or **mixed-effects modeling** when appropriate for your downstream analysis.


## Exercises (Try this)

1. **Change one knob at a time**
   - Increase `BATCH_SD` (e.g., 0.2 → 1.0 → 1.8). Observe how the naïve–grouped gap grows.
   - Toggle `CONFOUNDED=True/False`. When does the gap almost vanish?
   - Switch `AFFECTED` between `"informative"` and `"all"`. Which case hurts robustness more?

2. **Different grouping granularity**
   - Increase `N_BATCHES`. Does more (smaller) batches change the gap?
   - Try **Leave-One-Batch-Out** (LOBO) by setting `n_splits = N_BATCHES` in `GroupKFold`.

3. **Model sensitivity**
   - Replace Logistic Regression with **LinearSVC**, **RandomForest**, or **XGBoost**. Which model is most/least sensitive to batch leakage?

4. **Feature stability**
   - Compute top-k features by |Cohen’s d| on raw vs. batch-demeaned data and measure **Precision@k** overlap.

5. **Reporting**
   - Summarize in 3–5 sentences what setting(s) lead to the largest leakage and how grouped CV fixes it.


In [ ]:
from sklearn.model_selection import StratifiedKFold, GroupKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

def make_dataset_with_random_effects(
    n_samples=N_SAMPLES,
    n_inform=N_INFORM,
    n_noise=N_NOISE,
    n_classes=N_CLASSES,
    class_sep=CLASS_SEP,
    n_batches=N_BATCHES,
    batch_sd=0.0,
    confounded=False,
    affected=AFFECTED,
    seed=RANDOM_SEED,
):
    cfg = DatasetConfig(
        n_samples=n_samples,
        n_informative=n_inform,
        n_pseudo=0,
        n_noise=n_noise,
        n_classes=n_classes,
        class_sep=class_sep,
        feature_naming="prefixed",
        random_state=seed,
    )
    X0, y0, _ = generate_dataset(cfg, return_dataframe=True)
    inf_cols = [c for c in X0.columns if c.startswith("i")]
    noise_cols = [c for c in X0.columns if c.startswith("n")]
    feat_cols = inf_cols + noise_cols

    batches = assign_batches(y0, n_batches=n_batches, confounded=confounded, p_major=0.75, seed=seed)
    X0["batch_id"] = batches

    rng_local = np.random.default_rng(seed)
    shifts = rng_local.normal(0.0, batch_sd, size=n_batches)

    if affected == "informative":
        cols_to_shift = inf_cols
    else:
        cols_to_shift = feat_cols

    X_shift = X0.copy()
    for b in range(n_batches):
        mask = X_shift["batch_id"].values == b
        X_shift.loc[mask, cols_to_shift] = X_shift.loc[mask, cols_to_shift] + shifts[b]

    return X_shift, y0, feat_cols, batches


def eval_cv_gap(batch_sd_grid=(0.0, 0.5, 1.0, 1.5), confounded=True, seed=RANDOM_SEED):
    rows = []
    for sd in batch_sd_grid:
        Xs, ys, fcols, groups = make_dataset_with_random_effects(batch_sd=sd, confounded=confounded, seed=seed)
        X_mat = Xs[fcols].values
        y_arr = ys.values

        pipe = Pipeline([("scaler", StandardScaler()),
                         ("clf", LogisticRegression(max_iter=1000, random_state=seed))])

        cv_naive   = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        cv_grouped = GroupKFold(n_splits=min(len(np.unique(groups)), 5))

        scoring = {"bal_acc": "balanced_accuracy", "roc_auc": "roc_auc"}

        res_naive = cross_validate(pipe, X_mat, y_arr, cv=cv_naive, scoring=scoring, n_jobs=-1)
        res_group = cross_validate(pipe, X_mat, y_arr, cv=cv_grouped, groups=groups, scoring=scoring, n_jobs=-1)

        rows.append({
            "batch_sd": sd,
            "confounded": confounded,
            "bal_acc_naive": res_naive["test_bal_acc"].mean(),
            "bal_acc_group": res_group["test_bal_acc"].mean(),
            "roc_auc_naive": res_naive["test_roc_auc"].mean(),
            "roc_auc_group": res_group["test_roc_auc"].mean(),
        })
    return pd.DataFrame(rows).sort_values("batch_sd")

# Quick demo
df_sweep = eval_cv_gap(batch_sd_grid=[0.0, 0.4, 0.8, 1.2, 1.6], confounded=True)
display(df_sweep.round(3))


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(df_sweep["batch_sd"], df_sweep["bal_acc_naive"], marker="o", label="naive (bal_acc)")
ax.plot(df_sweep["batch_sd"], df_sweep["bal_acc_group"], marker="o", label="grouped (bal_acc)")
ax.set_xlabel("Batch SD")
ax.set_ylabel("Balanced accuracy (CV)")
ax.set_title("Effect of batch strength on naïve vs. grouped CV")
ax.legend()
plt.tight_layout()
plt.show()


### Leave-One-Batch-Out (LOBO)

For `n_splits = N_BATCHES`, `GroupKFold` becomes LOBO. This is a strong out-of-batch estimate when batches are a realistic deployment boundary (e.g., new day / new plate / new center).


In [ ]:
from sklearn.model_selection import GroupKFold, cross_val_score

X_mat = X[feature_cols].values
y_arr = y.values
groups = X["batch_id"].values

pipe = Pipeline([("scaler", StandardScaler()),
                 ("clf", LogisticRegression(max_iter=1000, random_state=RANDOM_SEED))])

cv_lobo = GroupKFold(n_splits=N_BATCHES)
scores = cross_val_score(pipe, X_mat, y_arr, cv=cv_lobo, groups=groups, scoring="balanced_accuracy", n_jobs=-1)
print(f"LOBO balanced accuracy: mean={scores.mean():.3f} ± {scores.std(ddof=1):.3f}")


## Beyond LOBO: handling batches without information leakage

LOBO shows the *evaluation* side. Next we contrast two preprocessing strategies:

1) **Leaky global batch demeaning** (WRONG): remove per-batch means using the **entire dataset** before CV.
2) **Fold-aware batch demeaning** (CORRECT): compute batch means **on the training fold only**, apply them to the test fold (unseen batches get no shift).

Goal: see how leakage inflates scores and confirm the safe alternative.


## Quick Takeaways

- **Random effects create structure** (batch/subject-specific offsets) that can inflate both **effect sizes** and **naïve CV** scores.
- If batches are **confounded** with the class, naïve CV becomes **optimistic**; **GroupKFold** approximates out-of-batch generalization.
- A simple **per-batch demeaning** shrinks inflated effect sizes and is a useful EDA probe (not a full correction).
- Always align your **CV scheme with the deployment boundary** (new day/plate/center/subject ⇒ grouped CV or LOBO).


## Common pitfalls (and how to avoid them)

**1) Wrong CV scheme**
- *Pitfall:* Using `StratifiedKFold` (or plain `KFold`) when batches/subjects exist → **leakage**.
- *Fix:* Use `GroupKFold` or `LeaveOneGroupOut` (LOBO). If available in your sklearn version, prefer `StratifiedGroupKFold` to keep class balance across grouped folds.

**2) Preprocessing before the split**
- *Pitfall:* Fitting scaler/imputer/PCA/feature selection on the **entire dataset** prior to CV.
- *Fix:* Put **all preprocessing inside a `Pipeline`** and let CV fit it **only on the training folds**.

**3) Batch demeaning as a “correction”**
- *Pitfall:* Applying per-batch demeaning on the full data and then doing CV → still leaks test-fold info.
- *Fix:* Treat per-batch demeaning as **EDA only** in this notebook. If you must use it in modeling, implement a **group-aware transformer** that fits means on the train fold and applies them to train/test separately.

**4) Using `batch_id` as a feature**
- *Pitfall:* Including `batch_id` in `X` → the model learns batch labels, not biology.
- *Fix:* Keep `batch_id` **only for grouping**, never as a predictor.

**5) Ignoring class imbalance within groups**
- *Pitfall:* `GroupKFold` does not stratify; some folds can be skewed.
- *Fix:* Check fold distributions. Use `StratifiedGroupKFold` if available, or adjust `class_weight='balanced'` and use **balanced accuracy**.

**6) Tuning vs. evaluation mismatch**
- *Pitfall:* Hyperparameter search with naïve CV, but final reporting with grouped CV (or vice versa).
- *Fix:* **Align your selection CV with your evaluation CV** (both grouped). For reliable performance estimates, use **nested CV** with groups.

**7) Too few batches**
- *Pitfall:* Very small number of groups → high variance estimates; tempting to revert to naïve CV.
- *Fix:* Prefer **LOBO** (one batch per test fold) and **report uncertainty** (mean ± SD). Consider collecting more batches if possible.

**8) Data augmentation / resampling leakage**
- *Pitfall:* Over/under-sampling, SMOTE, or augmentations applied before CV.
- *Fix:* Wrap any resampling step **inside the Pipeline** and ensure it runs **only on the training fold**.

**9) Feature engineering outside the split**
- *Pitfall:* Computing interactions/filters on the full dataset.
- *Fix:* Same rule: **inside the Pipeline**.

**10) Not checking confounding**
- *Pitfall:* Assuming batches are harmless.
- *Fix:* Always inspect `crosstab(batch, class)` and compute a dependence measure (e.g., **Cramér’s V**). Large values → expect naïve > grouped gap.

**11) Reporting the wrong number**
- *Pitfall:* Quoting training metrics or a single lucky split.
- *Fix:* Report **CV means ± SD**, prefer **balanced accuracy** / **ROC-AUC** over plain accuracy when classes are imbalanced.

**12) Uncontrolled randomness**
- *Pitfall:* Mixed RNG states across NumPy/sklearn; irreproducible results.
- *Fix:* Set `random_state` (model, CV shuffles, dataset generator) and log versions/seeds.

---

### Quick self-check (before you trust the numbers)
- ✅ `GroupKFold`/LOBO (or `StratifiedGroupKFold`) used for **both tuning and evaluation**
- ✅ All preprocessing inside a **Pipeline**
- ✅ `batch_id` used **only** for grouping, not as a feature
- ✅ **Naïve vs. grouped** gap inspected; Cramér’s V computed
- ✅ Results reported as **mean ± SD** over folds; code is **seeded** and versions logged


In [ ]:
import numpy as np
import pandas as pd

# --- 1) Quantify confounding between batch and class (Cramér's V) ---

def cramers_v(table: pd.DataFrame) -> float:
    obs = table.values.astype(float)
    n = obs.sum()
    if n == 0:
        return 0.0
    row = obs.sum(axis=1, keepdims=True)
    col = obs.sum(axis=0, keepdims=True)
    expected = row @ col / n
    # Avoid division by zero for empty cells
    mask = expected > 0
    chi2 = ((obs - expected)[mask] ** 2 / expected[mask]).sum()
    r, k = obs.shape
    denom = n * (min(r - 1, k - 1))
    return float(np.sqrt(chi2 / denom)) if denom > 0 else 0.0

ct = pd.crosstab(X["batch_id"], y)
V = cramers_v(ct)
print("Contingency table (batch × class):")
display(ct)
print(f"Cramér's V = {V:.3f}  (rule of thumb: ~0.1 small, ~0.3 medium, ~0.5 large)")

# --- 2) Leakage guardrail based on CV results ---

# Expect 'summary' from Cell 12 to be present: columns = [scheme, metric, mean, std]
def get_mean(summary_df: pd.DataFrame, scheme: str, metric: str) -> float:
    sel = summary_df[(summary_df["scheme"] == scheme) & (summary_df["metric"] == metric)]
    return float(sel["mean"].iloc[0]) if not sel.empty else float("nan")

ba_naive = get_mean(summary, "naive", "balanced_accuracy")
ba_group = get_mean(summary, "grouped", "balanced_accuracy")
gap = ba_naive - ba_group

print(f"\nBalanced accuracy — naive: {ba_naive:.3f}, grouped: {ba_group:.3f}, gap: {gap:.3f}")

# Soft guardrail: flag suspicious optimism
THRESH = 0.08  # tweak for your course; 0.05–0.10 are common teaching thresholds
if np.isfinite(gap) and gap > THRESH:
    print("⚠️  Guardrail: Naïve CV is notably higher than grouped CV. Likely batch leakage.")
else:
    print("✓ Guardrail: No substantial naïve > grouped gap detected.")


## Exercises

1) **One knob at a time**
- Sweep `BATCH_SD` (e.g., 0.0 → 0.4 → 0.8 → 1.2 → 1.6). Track the naive–grouped gap.
- Toggle `CONFOUNDED=True/False`. When does the gap nearly vanish?
- Switch `AFFECTED` between `"informative"` and `"all"`. Which hurts generalization more?

2) **Granularity of groups**
- Increase `N_BATCHES`. Does having many small batches change the gap?
- Try **LOBO** (`GroupKFold(n_splits=N_BATCHES)`) to simulate strict out-of-batch testing.

3) **Model sensitivity**
- Replace Logistic Regression with `LinearSVC`, `RandomForestClassifier`, or `XGBoost`.
- Which model is most sensitive to batch leakage in your setup?

4) **Feature stability**
- Rank top-k by |Cohen’s d| before vs. after per-batch demeaning and compute **Precision@k** overlap.

5) **Short report (5–8 sentences)**
- Describe the settings that produced the largest leakage and how grouped CV mitigated it.


## Next Steps

- **Pseudo-signals (03c):** Move from random intercepts to structured artifacts (hidden subclasses / pseudo features).
- **Correlated features (03d):** Study redundancy and multicollinearity; compare L1, Elastic Net, and tree-based importances.
- **Advanced handling (beyond this notebook):**
  - Mixed-effects models (random intercepts/slopes) for repeated measures.
  - Batch-aware preprocessing/normalization if domain-appropriate.
  - Stability analysis of selected features across grouped splits.